In [ ]:
from resnet import ResNet,Bottleneck,BasicBlock
import torch
import torch.nn as nn
import numpy as np
from torchvision import transforms
from torch.utils.data.sampler import SequentialSampler
import sys
sys.path.append("../_code")
from color_lib import RGBmean,RGBstdv
from Reader import ImageReader
from Utils import norml2
from similarity_ops import *
from image_ops import *
import os
from glob import glob
import random
from types import MethodType
import pickle
import matplotlib as mpt
import torch.nn.functional as F

In [ ]:
import pretrainedmodels
from powerful_benchmarker import architectures
from powerful_benchmarker import datasets
from pytorch_metric_learning.utils import common_functions as c_f
from pytorch_metric_learning.testers import GlobalEmbeddingSpaceTester
import torch
from torchvision import transforms
import powerful_benchmarker
import pytorch_metric_learning
import numpy as np
import os
from PIL import Image
print("powerful_benchmarker.__version__", powerful_benchmarker.__version__)
print("pytorch_metric_learning.__version__", pytorch_metric_learning.__version__)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

# get colormap
ncolors = 256
color_array = plt.get_cmap('jet')(range(ncolors))
# change alpha values
#color_array[:,-1] = np.linspace(0.0,1.0,ncolors)
#color_array[127:] = [0,0,0,0]

# create a colormap object
map_object = LinearSegmentedColormap.from_list(name='jet_alpha',colors=color_array)

# register this new colormap with matplotlib
plt.register_cmap(cmap=map_object)

In [4]:
import PIL
class ConvertToBGR(object):
    """
    Converts a PIL image from RGB to BGR
    """

    def __init__(self):
        pass

    def __call__(self, img):
        r, g, b = img.split()
        img = Image.merge("RGB", (b, g, r))
        return img

    def __repr__(self):
        return "{}()".format(self.__class__.__name__)


class Multiplier(object):
    def __init__(self, multiple):
        self.multiple = multiple

    def __call__(self, img):
        return img*self.multiple

    def __repr__(self):
        return "{}(multiple={})".format(self.__class__.__name__, self.multiple)

In [26]:
Data='CAR'
tra_or_val = 'val'

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

#model_epshn = torch.load('pretrained_model/'+Data+'/ArcFace/model.pth')

#model_triplet = torch.load('pretrained_model/'+Data+'/Triplet/model.pth')
#model_arcface = torch.load('./_result/'+Data+'/Arcface/model.pth')
# state_dict_npairs = torch.load('../_result_vis/CUB_R18_Npair/0/model.pth').state_dict()
# state_dict_nn = torch.load('../_result_vis/CUB_R18_G16/0/model.pth').state_dict()

output_folder = os.path.join(Data,'PCA',tra_or_val,'top_results')
#output_folder = os.path.join(Data,tra_or_val,'top_results')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

model_name = 'bninception'
model_epshn = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')
model_epshn.last_linear = nn.Linear(1024,128)
    
trunk=torch.load("../fair_exp/experiments/ArcFace/Test50_50_Partitions4_0/saved_models/trunk_best.pth")
embedder=torch.load("../fair_exp/experiments/ArcFace/Test50_50_Partitions4_0/saved_models/embedder_best.pth")

# trunk=torch.load("../fair_exp/experiments/ArcFace_CUB/Test50_50_Partitions4_0/saved_models/trunk_best.pth")
# embedder=torch.load("../fair_exp/experiments/ArcFace_CUB/Test50_50_Partitions4_0/saved_models/embedder_best.pth")

trunk.update(embedder)
trunk.pop('net.0.weight')
trunk.pop('net.0.bias')
model_epshn.load_state_dict(trunk)    

#data_dict = torch.load('CUB_toy.pth')
data_dict = torch.load('/pless_nfs/home/datasets/'+Data+'/data_dict_emb.pth')
# src='/pless_nfs/home/datasets/'+Data+'/data_e/'##
# data_dict = {p:{os.path.basename(d):glob(d+'/*.jpg') for d in glob(src+p+'/*')} for p in ['tra', 'val']}

embed_size=128

data_transforms = transforms.Compose([ConvertToBGR(),
                                      transforms.Resize(454),
                                      transforms.CenterCrop(454),
                                      transforms.ToTensor(),
                                      Multiplier(255),
                                      transforms.Normalize(mean = [104, 117, 128], 
                                                     std = [1, 1, 1])])

dsets = ImageReader(data_dict[tra_or_val], data_transforms)

model_epshn = model_epshn.cuda()
model_epshn.train(False)

#model_triplet = model_triplet.cuda()
#model_triplet = model_triplet

# model_arcface.avgpool = model_npairs.avgpool
# model_arcface = model_arcface.cuda()
# model_arcface.train(False)

BNInception(
  (conv1_7x7_s2): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (conv1_7x7_s2_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1_relu_7x7): ReLU(inplace=True)
  (pool1_3x3_s2): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=0, dilation=(1, 1), ceil_mode=True)
  (conv2_3x3_reduce): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
  (conv2_3x3_reduce_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2_relu_3x3_reduce): ReLU(inplace=True)
  (conv2_3x3): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_3x3_bn): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2_relu_3x3): ReLU(inplace=True)
  (pool2_3x3_s2): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=0, dilation=(1, 1), ceil_mode=True)
  (inception_3a_1x1): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1))
  (inception_3a_1x1_bn): BatchNorm

In [27]:
import importlib
from gradcam import GradCam
importlib.reload(sys.modules['gradcam'])
from gradcam import GradCam

In [28]:
grad_cam = GradCam(model=model_epshn, feature_module=model_epshn,target_layer_names=["global_pool"], use_cuda='True')

In [29]:
embeddings = []
def feature_map_hook(module, input, output):
    embeddings.append(input[0])

def register_embedding_hooks_vgg(model):
    for i,module in enumerate(model.children()):
        # If the layer has children of its own, recursively traverse
#         if list(module.children()):
#             register_embedding_hooks_vgg(module)
#         # If a pooling layer, register a hook
#         elif isinstance(module, nn.MaxPool2d):# or isinstance(module, nn.AdaptiveAvgPool2d):
#             print(f"Registering hook to Layer ({i}) {module}")
#             module.register_forward_hook(feature_map_hook)
        if isinstance(module, nn.AdaptiveAvgPool2d):
            print(f"Registering hook to Layer ({i}) {module}")
            module.register_forward_hook(feature_map_hook)
        # If a strided convolution layer, register a hook
        elif isinstance(module,nn.AvgPool2d):
            print(f"Registering hook to Layer ({i}) {module}")
            module.register_forward_hook(feature_map_hook)
#         elif isinstance(module, nn.Conv2d) and module.stride > (1,1):
#             print(f"Registering hook to Layer ({i}) {module}")
#             module.register_forward_hook(feature_map_hook)

def register_hooks_resnet(model):
    for i,module in enumerate(model.children()):
        if list(module.children()):
            print(f"Registering hook to Layer ({i})")
            module.register_forward_hook(feature_map_hook)
        elif isinstance(module, nn.MaxPool2d):# or isinstance(module, nn.AdaptiveAvgPool2d):
            print(f"Registering hook to Layer ({i}) {module}")
            module.register_forward_hook(feature_map_hook)
        elif isinstance(module, nn.AdaptiveAvgPool2d):
            print(f"Registering hook to Layer ({i}) {module}")
            module.register_forward_hook(feature_map_hook)
        elif isinstance(module,nn.AvgPool2d):
            print(f"Registering hook to Layer ({i}) {module}")
            module.register_forward_hook(feature_map_hook)

def activation_map(maps,input_size=224,interp_mode='bilinear'):
#     amap = []
#     amap = [F.interpolate(torch.sum(x,dim=1).unsqueeze(1), size=input_size,mode=interp_mode) for x in maps]
#     amap = torch.cat(amap, dim=1)
#     amap = torch.sum(amap,dim=1)

    amap = torch.norm(maps[0],dim=1).unsqueeze(1)
    amap = F.interpolate(amap, size=input_size,mode=interp_mode).squeeze()
    return amap


def embed(dsets,whichModel,imgsize=454,interp_mode='nearest'):
    global embeddings
    whichModel.train(False)
    dataLoader = torch.utils.data.DataLoader(dsets, batch_size=20, sampler=SequentialSampler(dsets), num_workers=12)
    V,M,L,S,M_f = [],[],[],[],[]
    try:
        fclayer = whichModel.last_linear
    except:
        fclayer = whichModel.fc
    for data in dataLoader:
        with torch.set_grad_enabled(False):
            inputs_bt, labels_bt = data['data'], data['label']
            fvec = whichModel(inputs_bt.cuda())
            fvec = (fvec.T/torch.norm(fvec,dim=1)).T
            fmap = embeddings[-1]
            final_map = fclayer(fmap.detach().clone().permute(0,2,3,1)).permute(0,3,1,2)
        V.extend(fvec.cpu().numpy())
        M.extend(fmap.cpu().numpy())
        M_f.extend(final_map.cpu().numpy())
        L.extend(labels_bt.numpy())
        S.extend(activation_map([embeddings[-1]],input_size=imgsize,interp_mode=interp_mode).cpu().numpy())
        embeddings = []
    maps_epshn = np.moveaxis(np.array(M),1,3)
    feats_epshn = np.array(V)
    labels = np.array(L)
    act_map = np.array(S)
    maps_final = np.moveaxis(np.array(M_f),1,3)
    return feats_epshn, maps_epshn,act_map, labels,maps_final

In [7]:
def get_vector(Data,loss,tra_or_val = 'val',imgsize=454):
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"]="1"
    dataname = ''
    if Data == 'CUB':
        dataname = '_CUB'
    model_name = 'bninception'
    model_epshn = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')
    model_epshn.last_linear = nn.Linear(1024,128)

    # trunk=torch.load("../fair_exp/experiments/ArcFace/Test50_50_Partitions4_0/saved_models/trunk_best36.pth")
    # embedder=torch.load("../fair_exp/experiments/ArcFace/Test50_50_Partitions4_0/saved_models/embedder_best36.pth")
 
    trunk=torch.load("../fair_exp/experiments/"+loss+dataname+"/Test50_50_Partitions4_0/saved_models/trunk_best.pth")
    embedder=torch.load("../fair_exp/experiments/"+loss+dataname+"/Test50_50_Partitions4_0/saved_models/embedder_best.pth")

    trunk.update(embedder)
    trunk.pop('net.0.weight')
    trunk.pop('net.0.bias')
    model_epshn.load_state_dict(trunk)  
    #model_arcface = torch.load('./_result/'+Data+'/Arcface/model.pth')
    # state_dict_npairs = torch.load('../_result_vis/CUB_R18_Npair/0/model.pth').state_dict()
    # state_dict_nn = torch.load('../_result_vis/CUB_R18_G16/0/model.pth').state_dict()
    data_dict = torch.load('/pless_nfs/home/datasets/'+Data+'/data_dict_emb.pth')
    # src='/pless_nfs/home/datasets/'+Data+'/data_e/'##
    # data_dict = {p:{os.path.basename(d):glob(d+'/*.jpg') for d in glob(src+p+'/*')} for p in ['tra', 'val']}

    embed_size=256

    data_transforms = transforms.Compose([ConvertToBGR(),
                                          transforms.Resize(454),
                                          transforms.CenterCrop(454),
                                          transforms.ToTensor(),
                                          Multiplier(255),
                                          transforms.Normalize(RGBmean['Imagenet'], RGBstdv['Imagenet'])])

    dsets = ImageReader(data_dict[tra_or_val], data_transforms)

    model_epshn = model_epshn.cuda()
    model_epshn.train(False)
    feats_epshn, maps_epshn, labels, epshn_weighs = embed(dsets,model_epshn,0,imgsize=454)
    featsavepath = os.path.join(Data,'savedata',tra_or_val,loss)
    if not os.path.exists(featsavepath):
        os.makedirs(featsavepath)
#     torch.save(feats_epshn,featsavepath+'feats.pth')
#     torch.save(maps_epshn,featsavepath+'maps.pth')
#     torch.save(labels,featsavepath+'labels.pth')
    return feats_epshn,maps_epshn,labels

In [11]:
def get_vector_phase(phase='',tra_or_val = 'val',imgsize=454):
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"]="1"
    dataname = ''
    if Data == 'CUB':
        dataname = '_CUB'
    model_name = 'bninception'
    model_epshn = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')
    model_epshn.last_linear = nn.Linear(1024,128)

    # trunk=torch.load("../fair_exp/experiments/ArcFace/Test50_50_Partitions4_0/saved_models/trunk_best36.pth")
    # embedder=torch.load("../fair_exp/experiments/ArcFace/Test50_50_Partitions4_0/saved_models/embedder_best36.pth")
 
    trunk=torch.load("../fair_exp/experiments/training_process_CUB_ArcFace/trunk_best"+str(phase)+".pth")
    embedder=torch.load("../fair_exp/experiments/training_process_CUB_ArcFace/embedder_best"+str(phase)+".pth")

    trunk.update(embedder)
    trunk.pop('net.0.weight')
    trunk.pop('net.0.bias')
    model_epshn.load_state_dict(trunk)  
    #model_arcface = torch.load('./_result/'+Data+'/Arcface/model.pth')
    # state_dict_npairs = torch.load('../_result_vis/CUB_R18_Npair/0/model.pth').state_dict()
    # state_dict_nn = torch.load('../_result_vis/CUB_R18_G16/0/model.pth').state_dict()
    data_dict = torch.load('/pless_nfs/home/datasets/'+Data+'/data_dict_emb.pth')
    # src='/pless_nfs/home/datasets/'+Data+'/data_e/'##
    # data_dict = {p:{os.path.basename(d):glob(d+'/*.jpg') for d in glob(src+p+'/*')} for p in ['tra', 'val']}

    embed_size=256

    data_transforms = transforms.Compose([ConvertToBGR(),
                                          transforms.Resize(454),
                                          transforms.CenterCrop(454),
                                          transforms.ToTensor(),
                                          Multiplier(255),
                                          transforms.Normalize(RGBmean['Imagenet'], RGBstdv['Imagenet'])])

    dsets = ImageReader(data_dict[tra_or_val], data_transforms)

    model_epshn = model_epshn.cuda()
    model_epshn.train(False)
    feats_epshn, maps_epshn, labels, epshn_weighs = embed(dsets,model_epshn,0,imgsize=454)
    #featsavepath = os.path.join(Data,'savedata',tra_or_val,loss)
    #if not os.path.exists(featsavepath):
    #    os.makedirs(featsavepath)
#     torch.save(feats_epshn,featsavepath+'feats.pth')
#     torch.save(maps_epshn,featsavepath+'maps.pth')
#     torch.save(labels,featsavepath+'labels.pth')
    return feats_epshn,maps_epshn,labels

In [ ]:
from collections import OrderedDict

Data='CAR'
tra_or_val = 'val'

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

#model_epshn = torch.load('pretrained_model/'+Data+'/ArcFace/model.pth')

#model_triplet = torch.load('pretrained_model/'+Data+'/Triplet/model.pth')
#model_arcface = torch.load('./_result/'+Data+'/Arcface/model.pth')
# state_dict_npairs = torch.load('../_result_vis/CUB_R18_Npair/0/model.pth').state_dict()
# state_dict_nn = torch.load('../_result_vis/CUB_R18_G16/0/model.pth').state_dict()

output_folder = os.path.join(Data,'PCA_pooling_training',tra_or_val,'top_results')
#output_folder = os.path.join(Data,tra_or_val,'top_results')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

model_name = 'bninception'
model_ori =torch.load('./pretrained_model/CAR/ArcFace/model.pth')  

model_pca = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')
model_pca.last_linear = nn.Linear(1024,128)
    
# trunk=torch.load("../fair_exp/experiments/ArcFace/Test50_50_Partitions4_0/saved_models/trunk_best36.pth")
# embedder=torch.load("../fair_exp/experiments/ArcFace/Test50_50_Partitions4_0/saved_models/embedder_best36.pth")

trunk=torch.load("/pless_nfs/home/liuxiaotong2017/fair_exp/experiments/PCA/CAR_optim/ArcFace/PCA/CAR_optim/ArcFace/7/Test50_50_Partitions4_0/saved_models/trunk_best32.pth")
embedder=torch.load("/pless_nfs/home/liuxiaotong2017/fair_exp/experiments/PCA/CAR_optim/ArcFace/PCA/CAR_optim/ArcFace/7/Test50_50_Partitions4_0/saved_models/embedder_best32.pth")

fcparam = OrderedDict()
for key in embedder:
    if 'weight' in key:
        print(key)
        fcparam[key] = embedder[key][:,3*1024:]
    if 'bias' in key:
        print(key)
        fcparam[key] = embedder[key]

trunk.update(fcparam)
# trunk.pop('net.0.weight')
# trunk.pop('net.0.bias')
model_pca.load_state_dict(trunk)  
    
data_dict = torch.load('/pless_nfs/home/datasets/'+Data+'/data_dict_emb.pth')
# src='/pless_nfs/home/datasets/'+Data+'/data_e/'##
# data_dict = {p:{os.path.basename(d):glob(d+'/*.jpg') for d in glob(src+p+'/*')} for p in ['tra', 'val']}

embed_size=128

data_transforms = transforms.Compose([ConvertToBGR(),
                                      transforms.Resize(256),
                                      transforms.CenterCrop(256),
                                      transforms.ToTensor(),
                                      Multiplier(255),
                                      transforms.Normalize(RGBmean['Imagenet'], RGBstdv['Imagenet'])])

dsets = ImageReader(data_dict[tra_or_val], data_transforms)

model_ori = model_ori.cuda()
model_ori.train(False)

model_pca = model_pca.cuda()
model_pca.train(False)

#model_triplet = model_triplet.cuda()
#model_triplet = model_triplet

# model_arcface.avgpool = model_npairs.avgpool
# model_arcface = model_arcface.cuda()
# model_arcface.train(False)

In [9]:
feats = []
maps = []
lables = []
# for Data in ['CAR']:
#     for loss in ['ArcFace','Triplet','ProxyNCA','MS']:
    #for loss in ['MS']:
for phase in [0,2,4,6,12]:
    print(phase,'start!')
    feats_epshn,maps_epshn,labels = get_vector_phase(phase,tra_or_val = 'tra', imgsize=454)
    feats.append(feats_epshn)
    maps.append(maps_epshn)

0 start!


NameError: name 'get_vector_phase' is not defined

In [9]:
register_hooks_resnet(model_epshn)
feats_epshn, maps_epshn,act_map, labels,_ = embed(dsets,model_epshn,imgsize=256,interp_mode = 'nearest')

Registering hook to Layer (3) MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=0, dilation=(1, 1), ceil_mode=True)
Registering hook to Layer (10) MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=0, dilation=(1, 1), ceil_mode=True)
Registering hook to Layer (29) AvgPool2d(kernel_size=3, stride=1, padding=1)
Registering hook to Layer (51) AvgPool2d(kernel_size=3, stride=1, padding=1)
Registering hook to Layer (70) MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=0, dilation=(1, 1), ceil_mode=True)
Registering hook to Layer (89) AvgPool2d(kernel_size=3, stride=1, padding=1)
Registering hook to Layer (111) AvgPool2d(kernel_size=3, stride=1, padding=1)
Registering hook to Layer (133) AvgPool2d(kernel_size=3, stride=1, padding=1)
Registering hook to Layer (155) AvgPool2d(kernel_size=3, stride=1, padding=1)
Registering hook to Layer (174) MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=0, dilation=(1, 1), ceil_mode=True)
Registering hook to Layer (193) AvgPool2d(kernel_size=3

In [32]:
maps_epshn.shape

(8131, 14, 14, 1024)

In [10]:
feats_epshn = torch.load('./WACV2022/Comparison/CUB/val/feats_epehn.pth')
act_map = torch.load('./WACV2022/Comparison/CUB/val/act_map.pth')
with open('./WACV2022/Comparison/CUB/val/CUB_val_conv_14*14.pth', 'rb') as handle:
    maps_epshn = pickle.load(handle)

In [31]:
feats_epshn = torch.load('./WACV2022/Comparison/CAR/val/feats_epehn.pth')
act_map = torch.load('./WACV2022/Comparison/CAR/val/act_map.pth')
with open('./WACV2022/Comparison/CAR/val/CAR_val_conv_14*14.pth', 'rb') as handle:
    maps_epshn = pickle.load(handle)

In [10]:
feats_epshn = torch.load('./WACV2022/Comparison/CAR/feats_epehn.pth')
act_map = torch.load('./WACV2022/Comparison/CAR/act_map.pth')
with open('./WACV2022/Comparison/CAR/CAR_tra_conv_14*14.pth', 'rb') as handle:
    maps_epshn = pickle.load(handle)

In [11]:
act_map.shape

(8054, 256, 256)

In [9]:
feats_epshn = torch.load('./WACV2022/Comparison/feats_epehn.pth')
act_map = torch.load('./WACV2022/Comparison/CUB/act_map.pth')
with open('/pless_nfs/home/liuxiaotong2017/../share_projects/CUB_14*14_conv/Arcface_CUB_tra_conv_14*14.pkl', 'rb') as f:
    maps_epshn = pickle.load(f)

In [33]:
labels=np.array(list(dsets.idx_to_class.values()))

In [34]:
data_transforms2 = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(256)])
dsets2 = ImageReader(data_dict[tra_or_val], data_transforms2)

In [14]:
def combine_image_and_heatmap(img,heatmap,lower_bound,upper_bound,greyscale = True):
    """
    Takes in a numpy array for an image and the similarity heatmap.
    Blends the two images together and returns a np array of the blended image.
    """
    register_colorbar()
    cmap = plt.get_cmap('jet_alpha')# colormap for the heatmap
    heatmap = heatmap - lower_bound 
    heatmap /= upper_bound
#     heatmap = cmap(np.max(heatmap)-heatmap)
    heatmap = cmap(heatmap)
    if np.max(heatmap) < 255.:
        heatmap *= 255

    heatmap_img = cv2.resize(heatmap,(256,256),interpolation = cv2.INTER_NEAREST)
    #heatmap_img = cv2.resize(heatmap,(256,256))
    bg = Image.fromarray(img.astype('uint8')).convert('RGB')
    if greyscale == True:
        matrix = (0.2, 0.5, 0.3, 0.0, 0.2, 0.5, 0.3, 0.0, 0.2, 0.5, 0.3, 0.0)
        bg = bg.convert('RGB',matrix)
    fg = Image.fromarray(heatmap_img.astype('uint8')).convert('RGB')
    outIm = np.array(Image.blend(bg,fg,alpha=0.5))
    return outIm


In [15]:
def combine_image_and_heatmap_rgb(img,heatmap,greyscale = True):
    """
    Takes in a numpy array for an image and the similarity heatmap.
    Blends the two images together and returns a np array of the blended image.
    """
    cmap = plt.get_cmap('jet')
    heatmap = cmap(np.max(heatmap)-heatmap)
    if np.max(heatmap) < 255.:
        heatmap *= 255
    #heatmap_img = cv2.resize(heatmap,(256,256),interpolation = cv2.INTER_NEAREST)
    heatmap_img = cv2.resize(heatmap,(256,256),interpolation = cv2.INTER_NEAREST)
    bg = Image.fromarray(img.astype('uint8')).convert('RGB')
    if greyscale == True:
        matrix = (0.2, 0.5, 0.3, 0.0, 0.2, 0.5, 0.3, 0.0, 0.2, 0.5, 0.3, 0.0)
        bg = bg.convert('RGB',matrix)
    fg = Image.fromarray(heatmap_img.astype('uint8')).convert('RGB')
    outIm = np.array(Image.blend(bg,fg,alpha=0.5))
    return outIm


In [16]:
def combine_image_and_heatmap_hsv(img,heatmap,norm,greyscale = True):
    """
    Takes in a numpy array for an image and the similarity heatmap.
    Blends the two images together and returns a np array of the blended image.
    """
    cmap='jet'
    heatmap = cv2.resize(heatmap,(256,256),interpolation = cv2.INTER_NEAREST )
    cmap = plt.get_cmap(cmap)
    heatmap = cmap(1-heatmap)
    heatmap *= 255.
    norm = cv2.resize(norm,(256,256),interpolation = cv2.INTER_NEAREST )
    norm = norm-norm.min()
    norm /= norm.max()
    heatmap[:,:,3] = norm*255.0
    bg = Image.fromarray(img.astype('uint8')).convert('RGB')
    if greyscale == True:
        matrix = (0.2, 0.5, 0.3, 0.0, 0.2, 0.5, 0.3, 0.0, 0.2, 0.5, 0.3, 0.0)
        bg = bg.convert('RGB',matrix)
    bg = bg.convert("RGBA")
    fg = Image.fromarray(heatmap.astype('uint8')).convert('RGBA')
    outIm = np.array(Image.alpha_composite(bg, fg))
    return outIm


In [17]:
def combine_image_and_heatmap_greyscale(norm,greyscale = True):
    """
    Takes in a numpy array for an image and the similarity heatmap.
    Blends the two images together and returns a np array of the blended image.
    """
    norm = norm-norm.min()
    norm = norm/norm.max()*255.0
    heatmap_img = cv2.resize(norm,(256,256),interpolation = cv2.INTER_NEAREST)
    #heatmap_img = cv2.resize(heatmap,(256,256))
#     bg = Image.fromarray(img.astype('uint8')).convert('RGB')
#     if greyscale == True:
#         matrix = (0.2, 0.5, 0.3, 0.0, 0.2, 0.5, 0.3, 0.0, 0.2, 0.5, 0.3, 0.0)
#         bg = bg.convert('RGB',matrix)
    fg = Image.fromarray(heatmap_img.astype('uint8'),'L')
    outIm = np.array(fg)
    return outIm

In [18]:
def combine_image_and_heatmap_nocmap(img,heatmap,greyscale = True):
    """
    Takes in a numpy array for an image and the similarity heatmap.
    Blends the two images together and returns a np array of the blended image.
    """
    heatmap_img = cv2.resize(heatmap,(256,256),interpolation = cv2.INTER_NEAREST)
    #heatmap_img = cv2.resize(heatmap,(256,256))
    bg = Image.fromarray(img.astype('uint8')).convert('RGB')
    if greyscale == True:
        matrix = (0.2, 0.5, 0.3, 0.0, 0.2, 0.5, 0.3, 0.0, 0.2, 0.5, 0.3, 0.0)
        bg = bg.convert('RGB',matrix)
    fg = Image.fromarray(heatmap_img.astype('uint8')).convert('RGB')
    outIm = np.array(Image.blend(bg,fg,alpha=0.5))
    return outIm


In [19]:
def grad_cam_visual(grad_cam,feats,allactMaps,queryInd,resultInds,labels):
        queryHms = []
        resultHms = []
        resultImgs = []
        query_input_Img = ImageReader.__getitem__(dsets2, queryInd)['data']
        queryImg = np.array(query_input_Img)
        query_input_Img = data_transforms(query_input_Img.copy()).unsqueeze(0)
        feat1 = feats[queryInd]
        norm1 = allactMaps[queryInd]

        for resultInd in resultInds:
            result_input_Img = ImageReader.__getitem__(dsets2, resultInd)['data']
            resultImg = np.array(result_input_Img)
            feat2 = feats[resultInd]
            norm2 = allactMaps[resultInd]
                    
            result_input_Img = data_transforms(result_input_Img.copy()).unsqueeze(0)
            sim1 = grad_cam(query_input_Img, feat2)
            sim2 = grad_cam(result_input_Img, feat1)
            im1_with_similarity = combine_image_and_heatmap(queryImg,sim1,sim1.min(),sim1.max())
            im2_with_similarity = combine_image_and_heatmap(resultImg,sim2,sim2.min(),sim2.max())

            queryHms.append(im1_with_similarity)
            resultHms.append(im2_with_similarity)
            if labels[resultInd] == labels[queryInd]:
                resultImgs.append(add_green_border(resultImg))
            else:
                resultImgs.append(add_red_border(resultImg))
            #nn_maps[idx].append(sim1)
        
        queryHmImgs = combine_horz(queryHms)
        resultHmImgs = combine_horz(resultHms)
        resultImgs = combine_horz(resultImgs)
        stackedImg = combine_vert([np.array(queryHmImgs),np.array(resultImgs),np.array(resultHmImgs)])
        leftImg = combine_vert([np.zeros((queryImg.shape)),queryImg,np.zeros((queryImg.shape))])
        outImg = combine_horz([np.array(leftImg),np.array(stackedImg)])
        return outImg

In [20]:
def make_sim_ims_abby(weighs,feats,computer_similarity_method,allMaps,queryInd,resultInds,labels,minVal,maxVal):
        queryHms = []
        resultHms = []
        resultImgs = []
        queryImg = np.array(ImageReader.__getitem__(dsets2, queryInd)['data'])
        conv1 = allMaps[queryInd].reshape((allMaps[0].shape[0]*allMaps[0].shape[1],allMaps[0].shape[2]))
        feat1 = feats[queryInd]

        for resultInd in resultInds:
            resultImg = np.array(ImageReader.__getitem__(dsets2, resultInd)['data'])
            conv2 = allMaps[resultInd].reshape((allMaps[0].shape[0]*allMaps[0].shape[1],allMaps[0].shape[2]))
            feat2 = feats[resultInd]
            kwargs = {"conv1" : conv1, "conv2" : conv2} 
            sim1,sim2 = computer_similarity_method(**kwargs)
            
            im1_with_similarity = combine_image_and_heatmap(queryImg,sim1,sim1.min(),sim1.max())
            im2_with_similarity = combine_image_and_heatmap(resultImg,sim2,sim2.min(),sim2.max())
            if resultInd == resultInds[0]:
                selfSimQ = sim1
                selfSimR = sim2

            queryHms.append(im1_with_similarity)
            resultHms.append(im2_with_similarity)
            if labels[resultInd] == labels[queryInd]:
                resultImgs.append(resultImg)
            else:
                resultImgs.append(add_red_border(resultImg))
            #nn_maps[idx].append(sim1)
        
        queryHmImgs = combine_horz(queryHms)
        resultHmImgs = combine_horz(resultHms)
        resultImgs = combine_horz(resultImgs)
        stackedImg = combine_vert([np.array(queryHmImgs),np.array(resultImgs),np.array(resultHmImgs)])
        leftImg = combine_vert([np.zeros((queryImg.shape)),queryImg,np.zeros((queryImg.shape))])
        outImg = combine_horz([np.array(leftImg),np.array(stackedImg)])
        return outImg, selfSimQ, selfSimR

In [21]:
def make_sim_ims_rgb(weighs,feats,computer_similarity_method,allMaps,queryInd,resultInds,labels,minVal,maxVal):
        queryHms = []
        resultHms = []
        resultImgs = []
        queryImg = np.array(ImageReader.__getitem__(dsets2, queryInd)['data'])
        conv1 = allMaps[queryInd].reshape((allMaps[0].shape[0]*allMaps[0].shape[1],allMaps[0].shape[2]))
        feat1 = feats[queryInd]

        for resultInd in resultInds:
            resultImg = np.array(ImageReader.__getitem__(dsets2, resultInd)['data'])
            conv2 = allMaps[resultInd].reshape((allMaps[0].shape[0]*allMaps[0].shape[1],allMaps[0].shape[2]))
            feat2 = feats[resultInd]
            kwargs = {"conv1" : conv1, "conv2" : conv2} 
            sim1,sim2 = computer_similarity_method(**kwargs)
            
            im1_with_similarity = combine_image_and_heatmap_nocmap(queryImg,sim1)
            im2_with_similarity = combine_image_and_heatmap_nocmap(resultImg,sim2)
            if resultInd == resultInds[0]:
                selfSimQ = sim1
                selfSimR = sim2

            queryHms.append(im1_with_similarity)
            resultHms.append(im2_with_similarity)
            if labels[resultInd] == labels[queryInd]:
                resultImgs.append(resultImg)
            else:
                resultImgs.append(add_red_border(resultImg))
            #nn_maps[idx].append(sim1)
        
        queryHmImgs = combine_horz(queryHms)
        resultHmImgs = combine_horz(resultHms)
        resultImgs = combine_horz(resultImgs)
        stackedImg = combine_vert([np.array(queryHmImgs),np.array(resultImgs),np.array(resultHmImgs)])
        leftImg = combine_vert([np.zeros((queryImg.shape)),queryImg,np.zeros((queryImg.shape))])
        outImg = combine_horz([np.array(leftImg),np.array(stackedImg)])
        return outImg, selfSimQ, selfSimR

In [39]:
def make_sim_ims_svdwhole_4loss(computer_similarity_method,allMaps,resultInds,labels,minVal,maxVal):        
        convlist1=[]
        convlist2=[]
        convlist3=[]
        convlist4=[]
        imglist=[]
        for resultInd in resultInds:
            resultImg = np.array(ImageReader.__getitem__(dsets2, resultInd)['data'])
            conv1 = allMaps[0][resultInd].reshape((allMaps[0][0].shape[0]*allMaps[0][0].shape[1],allMaps[0][0].shape[2]))
            conv2 = allMaps[1][resultInd].reshape((allMaps[0][0].shape[0]*allMaps[0][0].shape[1],allMaps[0][0].shape[2]))
            conv3 = allMaps[2][resultInd].reshape((allMaps[0][0].shape[0]*allMaps[0][0].shape[1],allMaps[0][0].shape[2]))
            conv4 = allMaps[3][resultInd].reshape((allMaps[0][0].shape[0]*allMaps[0][0].shape[1],allMaps[0][0].shape[2]))
            convlist1.append(conv1)
            convlist2.append(conv2)
            convlist3.append(conv3)
            convlist4.append(conv4)
            imglist.append(resultImg)
        kwargs1 = {"convlist" : convlist1}
        kwargs2 = {"convlist" : convlist2}
        kwargs3 = {"convlist" : convlist3}
        kwargs4 = {"convlist" : convlist4}
        reslist1 = computer_similarity_method(**kwargs1)
        reslist2 = computer_similarity_method(**kwargs2)
        reslist3 = computer_similarity_method(**kwargs3)
        reslist4 = computer_similarity_method(**kwargs4)
        resultHms1 = []
        resultHms2 = []
        resultHms3 = []
        resultHms4 = []
        for res,img in zip(reslist1,imglist):
            im_with_similarity = combine_image_and_heatmap_nocmap(img,res)
            resultHms1.append(im_with_similarity)
        for res,img in zip(reslist2,imglist):
            im_with_similarity = combine_image_and_heatmap_nocmap(img,res)
            resultHms2.append(im_with_similarity)
        for res,img in zip(reslist3,imglist):
            im_with_similarity = combine_image_and_heatmap_nocmap(img,res)
            resultHms3.append(im_with_similarity)
        for res,img in zip(reslist4,imglist):
            im_with_similarity = combine_image_and_heatmap_nocmap(img,res)
            resultHms4.append(im_with_similarity)
#         if labels[resultInd] == labels[queryInd]:
#             resultImgs.append(add_green_border(resultImg))
#         else:
#             resultImgs.append(add_red_border(resultImg))
        #nn_maps[idx].append(sim1)
        
        resultHmImgs1 = combine_horz(resultHms1)
        resultHmImgs2 = combine_horz(resultHms2)
        resultHmImgs3 = combine_horz(resultHms3)
        resultHmImgs4 = combine_horz(resultHms4)
        resultImgs = combine_horz(imglist)
        stackedImg = combine_vert([np.array(resultImgs),np.array(resultHmImgs1),np.array(resultHmImgs2),np.array(resultHmImgs3),np.array(resultHmImgs4)])
        return stackedImg

In [22]:
def make_sim_ims_reconsturction(feats,computer_similarity_method,allMaps,allactMaps,queryInd,resultInds,labels,mode = None,dim=0):
        queryHms = []
        resultHms = []
        resultImgs = []
        queryImg = np.array(ImageReader.__getitem__(dsets2, queryInd)['data'])
        conv1 = allMaps[queryInd].reshape((allMaps[0].shape[0]*allMaps[0].shape[1],allMaps[0].shape[2]))
        feat1 = feats[queryInd]
        norm1 = allactMaps[queryInd]

        for resultInd in resultInds:
            resultImg = np.array(ImageReader.__getitem__(dsets2, resultInd)['data'])
            conv2 = allMaps[resultInd].reshape((allMaps[0].shape[0]*allMaps[0].shape[1],allMaps[0].shape[2]))
            feat2 = feats[resultInd]
            norm2 = allactMaps[resultInd]
            kwargs = {"conv1" : conv1, "conv2" : conv2}
            if mode == 'svd':
                sim1,sim2 = computer_similarity_method(**kwargs)
                im1_with_similarity = combine_image_and_heatmap_rgb(queryImg,sim1)
                im2_with_similarity = combine_image_and_heatmap_rgb(resultImg,sim2)
            elif mode == 'norm':
                im1_with_similarity = combine_image_and_heatmap_greyscale(norm1)
                im2_with_similarity = combine_image_and_heatmap_greyscale(norm2)
            elif mode == 'cmap':
                kwargs = {"conv1" : conv1, "conv2" : conv2,"dim" : dim}
                sim1,sim2 = computer_similarity_method(**kwargs)
                im1_with_similarity = combine_image_and_heatmap_cmap(queryImg,sim1)
                im2_with_similarity = combine_image_and_heatmap_cmap(resultImg,sim2)
            elif mode =='test':
                sim1,sim2 = computer_similarity_method(**kwargs)
                im1_with_similarity = combine_image_and_heatmap_cmap(queryImg,sim1)
                im2_with_similarity = combine_image_and_heatmap_cmap(resultImg,sim2)
            else:
                sim1,sim2 = computer_similarity_method(**kwargs)

                im1_with_similarity = combine_image_and_heatmap_hsv(queryImg,sim1,norm1)
                #im1_with_similarity = combine_image_and_heatmap(queryImg,sim1,sim1.min(),sim2.max())
                im2_with_similarity = combine_image_and_heatmap_hsv(resultImg,sim2,norm2)
            #im2_with_similarity = combine_image_and_heatmap(resultImg,norm2,norm2.min(),norm2.max())

            queryHms.append(im1_with_similarity)
            resultHms.append(im2_with_similarity)
            if labels[resultInd] == labels[queryInd]:
                resultImgs.append(add_green_border(resultImg))
            else:
                resultImgs.append(add_red_border(resultImg))
            #nn_maps[idx].append(sim1)
        
        queryHmImgs = combine_horz(queryHms)
        resultHmImgs = combine_horz(resultHms)
        resultImgs = combine_horz(resultImgs)
        stackedImg = combine_vert([np.array(queryHmImgs),np.array(resultImgs),np.array(resultHmImgs)])
        leftImg = combine_vert([np.zeros((queryImg.shape)),queryImg,np.zeros((queryImg.shape))])
        outImg = combine_horz([np.array(leftImg),np.array(stackedImg)])
        return outImg

In [30]:
def make_sim_ims_svdwhole(feats,computer_similarity_method,allMaps,resultInds,labels,minVal,maxVal):        
        convlist=[]
        imglist=[]
        for resultInd in resultInds:
            resultImg = np.array(ImageReader.__getitem__(dsets2, resultInd)['data'])
            conv2 = allMaps[resultInd].reshape((allMaps[0].shape[0]*allMaps[0].shape[1],allMaps[0].shape[2]))
            feat2 = feats[resultInd]
            convlist.append(conv2)
            imglist.append(resultImg)
        kwargs = {"convlist" : convlist}
        reslist = computer_similarity_method(**kwargs)
        resultHms = []
        for res,img in zip(reslist,imglist):
            im_with_similarity = combine_image_and_heatmap_nocmap(img,res)
            resultHms.append(im_with_similarity)
#         if labels[resultInd] == labels[queryInd]:
#             resultImgs.append(add_green_border(resultImg))
#         else:
#             resultImgs.append(add_red_border(resultImg))
        #nn_maps[idx].append(sim1)
        
        resultHmImgs = combine_horz(resultHms)
        resultImgs = combine_horz(imglist)
        stackedImg = combine_vert([np.array(resultImgs),np.array(resultHmImgs)])
        return stackedImg

In [40]:
Data = 'CAR'
maps = []
for loss in ['ArcFace','Triplet','ProxyNCA','MultiSimilarity']:
    maps.append(torch.load(os.path.join(Data,'savedata',tra_or_val,loss)+'maps.pth'))
labels = torch.load(os.path.join(Data,'savedata',tra_or_val,'ArcFace')+'labels.pth')

In [33]:
Data = 'CUB'
data_dict = torch.load('/pless_nfs/home/datasets/'+Data+'/data_dict_emb.pth')
# src='/pless_nfs/home/datasets/'+Data+'/data_e/'##
# data_dict = {p:{os.path.basename(d):glob(d+'/*.jpg') for d in glob(src+p+'/*')} for p in ['tra', 'val']}

embed_size=256

data_transforms = transforms.Compose([ConvertToBGR(),
                                      transforms.Resize(227),
                                      transforms.CenterCrop(227),
                                      transforms.ToTensor(),
                                      Multiplier(255),
                                      transforms.Normalize(RGBmean['Imagenet'], RGBstdv['Imagenet'])])

data_transforms2 = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(256)])
dsets2 = ImageReader(data_dict[tra_or_val], data_transforms2)

In [17]:
Data = 'CAR'
loss = 'ArcFace'
feats_epshn = torch.load(os.path.join(Data,'savedata',tra_or_val,loss)+'feats.pth')
maps_epshn = torch.load(os.path.join(Data,'savedata',tra_or_val,loss)+'maps.pth')
labels = torch.load(os.path.join(Data,'savedata',tra_or_val,'ArcFace')+'labels.pth')

In [ ]:
import importlib
import sys
importlib.reload(sys.modules['similarity_ops'])
from similarity_ops import *

numResults = 20
computer_similarity_method = SVD_whole

output_folder = os.path.join(Data,'SVD_wholeclass_four_14',tra_or_val)
#output_folder = os.path.join(Data,tra_or_val,'top_results')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

epshn_maps = {}
#arcface_maps = {}
for idx in set(labels):
    sameClassInds = np.where(labels==idx)[0]
    outFilename = os.path.join(output_folder,str(idx)+'.jpg')
    
    res_map= make_sim_ims_svdwhole_4loss(computer_similarity_method,maps4,sameClassInds,labels,0,0)
    res_map.save(outFilename)
    print(outFilename)

In [23]:
from PIL import ImageDraw,ImageFont
def img_with_txt(txt):
    img = Image.new('RGB', (256, 256), color = 'black')
    draw = ImageDraw.Draw(img)
    w, h = img.size

    font = ImageFont.truetype("/usr/share/fonts/dejavu/DejaVuSerif-Bold.ttf", 40)
    text_w, text_h = draw.textsize(txt, font)

    draw.text((10, 150), txt, (255,255,255), font=font)
    
    return img

In [24]:
import importlib
import sys
importlib.reload(sys.modules['similarity_ops'])
importlib.reload(sys.modules['gradcam'])
from gradcam import GradCam
from similarity_ops import *

In [36]:
numResults = 20
computer_similarity_method_3 = SVD_reconstruction_thres
computer_similarity_method_2 = compute_spatial_similarity
computer_similarity_method_1 = combine_SVD

output_folder = os.path.join('WACV2022','Comparison','CAR','val')
#output_folder = os.path.join(Data,tra_or_val,'top_results')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

epshn_maps = {}
#arcface_maps = {}
for idx in range(0,5000,10):
#for idx in [2680]:
#for idx in range(0,5000,10):
    epshn_maps[idx] = []
    #arcface_maps[idx] = []
    
    epshn_feat = feats_epshn[idx]
    #arcface_feat = feats_arcface[idx]
    queryCls = labels[idx]
    sameClassInds = np.where(labels==queryCls)[0]
    outFilename = os.path.join(output_folder,'plus'+str(idx)+'.jpg')

#     # arcface dists
#     dists_arcface = [np.dot(arcface_feat/np.linalg.norm(arcface_feat),f/np.linalg.norm(f)) for f in feats_arcface]
#     dists_arcface = np.array(dists_arcface)
#     sortedInds_arcface = np.argsort(-dists_arcface)[:numResults]

    # epshn dists
    dists_epshn = [np.dot(epshn_feat/np.linalg.norm(epshn_feat),f/np.linalg.norm(f)) for f in feats_epshn]
    dists_epshn = np.array(dists_epshn)
    sortedInds_epshn= np.argsort(-dists_epshn)
    

    sortedInds_epshn_furthest = sortedInds_epshn[-numResults:]
    sortedInds_epshn_nearest = sortedInds_epshn[:numResults]
    sortedInds_epshn_same_furthest = sortedInds_epshn[labels[sortedInds_epshn] == labels[idx]][-10:]

    epshn_epshn_nearest_imgs1 = combine_horz([np.array(img_with_txt((str(queryCls)))),np.array(combine_horz([np.array(img_with_txt('%0.5f'%(d))) for d in dists_epshn[sortedInds_epshn_nearest]]))])
    epshn_nearest_map1, s1_arcface, s2_arcface = make_sim_ims_abby(0,feats_epshn,computer_similarity_method_2,maps_epshn,idx,sortedInds_epshn_nearest,labels,0,0)
    img0 = combine_vert([np.array(epshn_epshn_nearest_imgs1),np.array(epshn_nearest_map1)])
    
    epshn_epshn_nearest_imgs2 = combine_horz([np.zeros((256,256,3)),np.array(combine_horz([np.array(img_with_txt('%0.5f'%(d))) for d in dists_epshn[sortedInds_epshn_nearest]]))])
    epshn_nearest_map2, s1_arcface, s2_arcface = make_sim_ims_rgb(0,feats_epshn,computer_similarity_method_1,maps_epshn,idx,sortedInds_epshn_nearest,labels,0,0)
    img1 = combine_vert([np.array(epshn_epshn_nearest_imgs2),np.array(epshn_nearest_map2)])
    
    epshn_epshn_nearest_imgs3 = combine_horz([np.zeros((256,256,3)),np.array(combine_horz([np.array(img_with_txt('%0.5f'%(d))) for d in dists_epshn[sortedInds_epshn_nearest]]))])
    epshn_nearest_map3 = grad_cam_visual(grad_cam,feats_epshn,act_map,idx,sortedInds_epshn_nearest,labels)
    img2 = combine_vert([np.array(epshn_epshn_nearest_imgs2),np.array(epshn_nearest_map3)])
    
    epshn_epshn_nearest_imgs = combine_horz([np.zeros((256,256,3)),np.array(combine_horz([np.array(img_with_txt('%0.5f'%(d))) for d in dists_epshn[sortedInds_epshn_nearest]]))])
    epshn_nearest_map = make_sim_ims_reconsturction(feats_epshn,computer_similarity_method_3,maps_epshn,act_map,idx,sortedInds_epshn_nearest,labels,mode = None,dim=0)
    img3 = combine_vert([np.array(epshn_epshn_nearest_imgs),np.array(epshn_nearest_map)])
 
    outImg = combine_vert([np.array(img2),np.zeros((10,epshn_nearest_map2.width)),np.array(img0),np.zeros((10,epshn_nearest_map2.width)),np.array(img1),np.zeros((10,epshn_nearest_map2.width)),np.array(img3)])
    outImg.save(outFilename)
    print(outFilename)

WACV2022/Comparison/CAR/val/plus0.jpg
WACV2022/Comparison/CAR/val/plus10.jpg
WACV2022/Comparison/CAR/val/plus20.jpg
WACV2022/Comparison/CAR/val/plus30.jpg
WACV2022/Comparison/CAR/val/plus40.jpg
WACV2022/Comparison/CAR/val/plus50.jpg
WACV2022/Comparison/CAR/val/plus60.jpg
WACV2022/Comparison/CAR/val/plus70.jpg
WACV2022/Comparison/CAR/val/plus80.jpg
WACV2022/Comparison/CAR/val/plus90.jpg
WACV2022/Comparison/CAR/val/plus100.jpg
WACV2022/Comparison/CAR/val/plus110.jpg
WACV2022/Comparison/CAR/val/plus120.jpg
WACV2022/Comparison/CAR/val/plus130.jpg
WACV2022/Comparison/CAR/val/plus140.jpg
WACV2022/Comparison/CAR/val/plus150.jpg
WACV2022/Comparison/CAR/val/plus160.jpg
WACV2022/Comparison/CAR/val/plus170.jpg
WACV2022/Comparison/CAR/val/plus180.jpg
WACV2022/Comparison/CAR/val/plus190.jpg
WACV2022/Comparison/CAR/val/plus200.jpg
WACV2022/Comparison/CAR/val/plus210.jpg
WACV2022/Comparison/CAR/val/plus220.jpg
WACV2022/Comparison/CAR/val/plus230.jpg
WACV2022/Comparison/CAR/val/plus240.jpg
WACV2022/Co

In [33]:
tra_or_val='tra'

In [ ]:
numResults = 20
computer_similarity_method = combine_SVD

output_folder = os.path.join(Data,'training_14*14',tra_or_val)
#output_folder = os.path.join(Data,tra_or_val,'top_results')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
epshn_maps = maps[-1]
epshn_maps = {}
feats_epshn = feats[-1]
feats_0 = feats[0]
#arcface_maps = {}
for idx in range(0,5000,10):
#for idx in [10]:
#for idx in range(0,5000,10):
    epshn_maps[idx] = []
    #arcface_maps[idx] = []
    
    epshn_feat = feats_epshn[idx]
    epshn_feat_0  = feats_0[idx]
    #arcface_feat = feats_arcface[idx]
    queryCls = labels[idx]
    sameClassInds = np.where(labels==queryCls)[0]
    outFilename = os.path.join(output_folder,str(idx)+'.jpg')

#     # arcface dists
#     dists_arcface = [np.dot(arcface_feat/np.linalg.norm(arcface_feat),f/np.linalg.norm(f)) for f in feats_arcface]
#     dists_arcface = np.array(dists_arcface)
#     sortedInds_arcface = np.argsort(-dists_arcface)[:numResults]

    # epshn dists
    dists_epshn = [np.dot(epshn_feat/np.linalg.norm(epshn_feat),f/np.linalg.norm(f)) for f in feats_epshn]
    dists_epshn = np.array(dists_epshn)
    sortedInds_epshn= np.argsort(-dists_epshn)

    dists_0 = [np.dot(epshn_feat_0/np.linalg.norm(epshn_feat_0),f/np.linalg.norm(f)) for f in feats_0]
    dists_0 = np.array(dists_0)
    sortedInds_0= np.argsort(-dists_0)
    #sortedInds_epshn_furthest = sortedInds_epshn[-numResults:]
    sortedInds_epshn_nearest = sortedInds_epshn[:numResults]
    sortedInds_0_nearest = sortedInds_0[:numResults]
    
#     print(sortedInds_0_nearest)
#     print(sortedInds_epshn_nearest)
    
#     out = sortedInds_0_nearest[sortedInds_epshn_nearest[np.searchsorted(sortedInds_epshn_nearest,sortedInds_0_nearest)] != sortedInds_0_nearest]
#     print(out)

    reslist=[2519,2658,2554,2660,2474,2542,2704,2555,2767,45,2566,2677,2706,2507,31,37]
    #reslist = [2519]
#     for item in out:
#         if label[out]!=label[idx]:
#             print(item)
#             reslist.append(item)
    
    sortedInds_epshn_same_furthest = sortedInds_epshn[labels[sortedInds_epshn] == labels[idx]][-10:]
    
    epshn_epshn_nearest_imgs0 = combine_horz([np.zeros((256,256,3)),np.array(combine_horz([np.array(img_with_txt('%0.5f'%(d))) for d in dists_epshn[sortedInds_epshn_nearest]]))])
    epshn_nearest_map0, s1_arcface, s2_arcface = make_sim_ims_rgb(0,feats[0],computer_similarity_method,maps[0],idx,sortedInds_epshn_nearest,labels,0,0)
    img0 = combine_vert([np.array(epshn_epshn_nearest_imgs0),np.array(epshn_nearest_map0)])
    
    epshn_epshn_nearest_imgs1 = combine_horz([np.zeros((256,256,3)),np.array(combine_horz([np.array(img_with_txt('%0.5f'%(d))) for d in dists_epshn[sortedInds_epshn_nearest]]))])
    epshn_nearest_map1, s1_arcface, s2_arcface = make_sim_ims_rgb(0,feats[1],computer_similarity_method,maps[1],idx,sortedInds_epshn_nearest,labels,0,0)
    img1 = combine_vert([np.array(epshn_epshn_nearest_imgs1),np.array(epshn_nearest_map1)])
    
    epshn_epshn_nearest_imgs2 = combine_horz([np.zeros((256,256,3)),np.array(combine_horz([np.array(img_with_txt('%0.5f'%(d))) for d in dists_epshn[sortedInds_epshn_nearest]]))])
    epshn_nearest_map2, s1_arcface, s2_arcface = make_sim_ims_rgb(0,feats[2],computer_similarity_method,maps[2],idx,sortedInds_epshn_nearest,labels,0,0)
    img2 = combine_vert([np.array(epshn_epshn_nearest_imgs2),np.array(epshn_nearest_map2)])
    
    epshn_epshn_nearest_imgs3 = combine_horz([np.zeros((256,256,3)),np.array(combine_horz([np.array(img_with_txt('%0.5f'%(d))) for d in dists_epshn[sortedInds_epshn_nearest]]))])
    epshn_nearest_map3, s1_arcface, s2_arcface = make_sim_ims_rgb(0,feats[3],computer_similarity_method,maps[3],idx,sortedInds_epshn_nearest,labels,0,0)
    img3 = combine_vert([np.array(epshn_epshn_nearest_imgs3),np.array(epshn_nearest_map3)])
    
    epshn_epshn_nearest_imgs4 = combine_horz([np.zeros((256,256,3)),np.array(combine_horz([np.array(img_with_txt('%0.5f'%(d))) for d in dists_epshn[sortedInds_epshn_nearest]]))])
    epshn_nearest_map4, s1_arcface, s2_arcface = make_sim_ims_rgb(0,feats[4],computer_similarity_method,maps[4],idx,sortedInds_epshn_nearest,labels,0,0)
    img4 = combine_vert([np.array(epshn_epshn_nearest_imgs4),np.array(epshn_nearest_map4)])
    
    outImg = combine_vert([np.array(img0),np.zeros((10,epshn_nearest_map4.width)),np.array(img1),np.zeros((10,epshn_nearest_map4.width)),np.array(img2),np.zeros((10,epshn_nearest_map4.width)),np.array(img3),np.zeros((10,epshn_nearest_map4.width)),np.array(img4)])
    outImg.save(outFilename)
    print(outFilename)

In [59]:
labels[2519]

42

In [ ]:
numResults = 20
computer_similarity_method = combine_SVD

output_folder = os.path.join(Data,'PCA_training','combine_SVD',tra_or_val)
#output_folder = os.path.join(Data,tra_or_val,'top_results')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

pca_maps = {}
ori_maps = {}
for idx in range(0,5000,10):
#for idx in [120]:
#for idx in range(0,5000,10):
    pca_maps[idx] = []
    ori_maps[idx] = []
    
    pca_feat = feats_pca[idx]
    ori_feat = feats_ori[idx]
    queryCls = labels[idx]
    sameClassInds = np.where(labels==queryCls)[0]
    outFilename = os.path.join(output_folder,str(idx)+'.jpg')

    # arcface dists
    dists_ori = [np.dot(ori_feat/np.linalg.norm(ori_feat),f/np.linalg.norm(f)) for f in feats_ori]
    dists_ori = np.array(dists_ori)
    sortedInds_ori = np.argsort(-dists_ori)

    # epshn dists
    dists_pca = [np.dot(pca_feat/np.linalg.norm(pca_feat),f/np.linalg.norm(f)) for f in feats_pca]
    dists_pca = np.array(dists_pca)
    sortedInds_pca= np.argsort(-dists_pca)
    
    sortedInds_pca_nearest = sortedInds_pca[:numResults]
    sortedInds_ori_nearest = sortedInds_ori[:numResults]
    
    ori_ori_imgs = combine_horz([np.zeros((256,256,3)),np.array(combine_horz([np.array(img_with_txt('%0.5f'%(d))) for d in dists_ori[sortedInds_ori_nearest]]))])
    ori_ori_map, s1_arcface, s2_arcface = make_sim_ims_rgb(0,feats_ori,computer_similarity_method,maps_ori,idx,sortedInds_ori_nearest,labels,0,0)
    img0 = combine_vert([np.array(ori_ori_imgs),np.array(ori_ori_map)])
    
    ori_pca_imgs = combine_horz([np.zeros((256,256,3)),np.array(combine_horz([np.array(img_with_txt('%0.5f'%(d))) for d in dists_ori[sortedInds_ori_nearest]]))])
    ori_pca_map, s1_arcface, s2_arcface = make_sim_ims_rgb(0,feats_pca,computer_similarity_method,maps_pca,idx,sortedInds_ori_nearest,labels,0,0)
    img1 = combine_vert([np.array(ori_pca_imgs),np.array(ori_pca_map)])

    pca_ori_imgs = combine_horz([np.zeros((256,256,3)),np.array(combine_horz([np.array(img_with_txt('%0.5f'%(d))) for d in dists_pca[sortedInds_pca_nearest]]))])
    pca_ori_map, s1_arcface, s2_arcface = make_sim_ims_rgb(0,feats_ori,computer_similarity_method,maps_ori,idx,sortedInds_pca_nearest,labels,0,0)
    img2 = combine_vert([np.array(pca_ori_imgs),np.array(pca_ori_map)])
    
    pca_pca_imgs = combine_horz([np.zeros((256,256,3)),np.array(combine_horz([np.array(img_with_txt('%0.5f'%(d))) for d in dists_pca[sortedInds_pca_nearest]]))])
    pca_pca_map, s1_arcface, s2_arcface = make_sim_ims_rgb(0,feats_pca,computer_similarity_method,maps_pca,idx,sortedInds_pca_nearest,labels,0,0)
    img3 = combine_vert([np.array(pca_pca_imgs),np.array(pca_pca_map)])
    
    outImg = combine_vert([np.array(img0),np.zeros((10,ori_ori_imgs.width)),np.array(img1),np.zeros((10,ori_ori_imgs.width)),np.array(img2),np.zeros((10,ori_ori_imgs.width)),np.array(img3)])
    outImg.save(outFilename)
    print(outFilename)

In [32]:
numResults = 20
computer_similarity_method = SVD_reconstruction_thres

#output_folder = os.path.join(Data,'PCA_training','reconstruction',tra_or_val)
output_folder = os.path.join('CUB_toy')
#output_folder = os.path.join(Data,tra_or_val,'top_results')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

epshn_maps = {}
for idx in range(0,5000,1):
#for idx in [120]:
#for idx in range(0,5000,10):
    epshn_maps[idx] = []
    
    epshn_feat = feats_epshn[idx]
    queryCls = labels[idx]
    sameClassInds = np.where(labels==queryCls)[0]
    outFilename = os.path.join(output_folder,str(idx)+'.jpg')


    dists_epshn = [np.dot(epshn_feat/np.linalg.norm(epshn_feat),f/np.linalg.norm(f)) for f in feats_epshn]
    dists_epshn = np.array(dists_epshn)
    sortedInds_epshn= np.argsort(-dists_epshn)
    
    sortedInds_epshn_nearest = sortedInds_epshn[:numResults]
    

    epshn_ori_imgs = combine_horz([np.zeros((256,256,3)),np.array(combine_horz([np.array(img_with_txt('%0.5f'%(d))) for d in dists_epshn[sortedInds_epshn_nearest]]))])
    #epshn_ori_map, s1_arcface, s2_arcface = make_sim_ims_reconsturction(0,feats_epshn,computer_similarity_method,maps_epshn,idx,sortedInds_epshn_nearest,labels,0,0)
    epshn_ori_map, s1_arcface, s2_arcface = make_sim_ims_reconsturction(feats_epshn,computer_similarity_method,maps_epshn,act_map,idx,sortedInds_epshn_nearest,labels)
    img2 = combine_vert([np.array(epshn_ori_imgs),np.array(epshn_ori_map)])
    
    
    img2.save(outFilename)
    print(outFilename)

KeyboardInterrupt: 

In [36]:
numResults = 20
computer_similarity_method = combine_SVD

output_folder = os.path.join(Data,'combine_SVD_14*14',tra_or_val)
#output_folder = os.path.join(Data,tra_or_val,'top_results')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

epshn_maps = {}
#arcface_maps = {}
for idx in range(1000,5000,10):
#for idx in [120]:
#for idx in range(0,5000,10):
    epshn_maps[idx] = []
    #arcface_maps[idx] = []
    
    epshn_feat = feats_epshn[idx]
    #arcface_feat = feats_arcface[idx]
    queryCls = labels[idx]
    sameClassInds = np.where(labels==queryCls)[0]
    outFilename = os.path.join(output_folder,str(idx)+'.jpg')

#     # arcface dists
#     dists_arcface = [np.dot(arcface_feat/np.linalg.norm(arcface_feat),f/np.linalg.norm(f)) for f in feats_arcface]
#     dists_arcface = np.array(dists_arcface)
#     sortedInds_arcface = np.argsort(-dists_arcface)[:numResults]

    # epshn dists
    dists_epshn = [np.dot(epshn_feat/np.linalg.norm(epshn_feat),f/np.linalg.norm(f)) for f in feats_epshn]
    dists_epshn = np.array(dists_epshn)
    sortedInds_epshn= np.argsort(-dists_epshn)
    

    sortedInds_epshn_furthest = sortedInds_epshn[-numResults:]
    sortedInds_epshn_nearest = sortedInds_epshn[:numResults]
    sortedInds_epshn_same_furthest = sortedInds_epshn[labels[sortedInds_epshn] == labels[idx]][-10:]
    
    epshn_epshn_furthest_imgs = combine_horz([np.zeros((256,256,3)),np.array(combine_horz([np.array(img_with_txt('%0.5f'%(d))) for d in dists_epshn[sortedInds_epshn_furthest]]))])
    epshn_furthest_map, s1_arcface, s2_arcface = make_sim_ims_rgb(0,feats_epshn,computer_similarity_method,maps_epshn,idx,sortedInds_epshn_furthest,labels,0,0)
    img0 = combine_vert([np.array(epshn_epshn_furthest_imgs),np.array(epshn_furthest_map)])
    
    epshn_epshn_nearest_imgs = combine_horz([np.zeros((256,256,3)),np.array(combine_horz([np.array(img_with_txt('%0.5f'%(d))) for d in dists_epshn[sortedInds_epshn_nearest]]))])
    epshn_nearest_map, s1_arcface, s2_arcface = make_sim_ims_rgb(0,feats_epshn,computer_similarity_method,maps_epshn,idx,sortedInds_epshn_nearest,labels,0,0)
    img1 = combine_vert([np.array(epshn_epshn_nearest_imgs),np.array(epshn_nearest_map)])
    
    epshn_epshn_samefurthest_imgs = combine_horz([np.zeros((256,256,3)),np.array(combine_horz([np.array(img_with_txt('%0.5f'%(d))) for d in dists_epshn[sortedInds_epshn_same_furthest]]))])
    epshn_samefurthest_map, s1_arcface, s2_arcface = make_sim_ims_rgb(0,feats_epshn,computer_similarity_method,maps_epshn,idx,sortedInds_epshn_same_furthest,labels,0,0)
    img2 = combine_vert([np.array(epshn_epshn_samefurthest_imgs),np.array(epshn_samefurthest_map)])
    
    outImg = combine_vert([np.array(img0),np.zeros((10,epshn_epshn_furthest_imgs.width)),np.array(img1),np.zeros((10,epshn_epshn_furthest_imgs.width)),np.array(img2)])
    outImg.save(outFilename)
    print(outFilename)

KeyboardInterrupt: 

In [ ]:
import importlib
import sys
importlib.reload(sys.modules['similarity_ops'])
from similarity_ops import *

numResults = 20
computer_similarity_method = SVD_whole

output_folder = os.path.join(Data,'SVD_wholeclass_14*14',tra_or_val)
#output_folder = os.path.join(Data,tra_or_val,'top_results')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

epshn_maps = {}
#arcface_maps = {}
for idx in set(labels):
    sameClassInds = np.where(labels==idx)[0]
    outFilename = os.path.join(output_folder,str(idx)+'.jpg')
    
    res_map= make_sim_ims_svdwhole(feats_epshn,computer_similarity_method,maps_epshn,sameClassInds,labels,0,0)
    res_map.save(outFilename)
    print(outFilename)

In [ ]:
import importlib
import sys
importlib.reload(sys.modules['similarity_ops'])
from similarity_ops import *


numResults = 20
computer_similarity_method = SVD_reconstruction_thres

output_folder = os.path.join(Data,'pair_consturction_14*14',tra_or_val)
output_fol
#output_folder = os.path.join(Data,tra_or_val,'top_results')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

epshn_maps = {}
#arcface_maps = {}
for idx in range(100,5000,10):
#for idx in [10]:
#for idx in range(0,5000,10):
    epshn_maps[idx] = []
    #arcface_maps[idx] = []
    
    epshn_feat = feats_epshn[idx]
    #arcface_feat = feats_arcface[idx]
    queryCls = labels[idx]
    sameClassInds = np.where(labels==queryCls)[0]
    outFilename = os.path.join(output_folder,str(idx)+'.jpg')

#     # arcface dists
#     dists_arcface = [np.dot(arcface_feat/np.linalg.norm(arcface_feat),f/np.linalg.norm(f)) for f in feats_arcface]
#     dists_arcface = np.array(dists_arcface)
#     sortedInds_arcface = np.argsort(-dists_arcface)[:numResults]

    # epshn dists
    dists_epshn = [np.dot(epshn_feat/np.linalg.norm(epshn_feat),f/np.linalg.norm(f)) for f in feats_epshn]
    dists_epshn = np.array(dists_epshn)
    sortedInds_epshn= np.argsort(-dists_epshn)
    

    sortedInds_epshn_furthest = sortedInds_epshn[-numResults:]
    sortedInds_epshn_nearest = sortedInds_epshn[:numResults]
    sortedInds_epshn_same_furthest = sortedInds_epshn[labels[sortedInds_epshn] == labels[idx]][-10:]
    
    epshn_epshn_furthest_imgs = combine_horz([np.zeros((256,256,3)),np.array(combine_horz([np.array(img_with_txt('%0.5f'%(d))) for d in dists_epshn[sortedInds_epshn_furthest]]))])
    epshn_furthest_map, s1_arcface, s2_arcface = make_sim_ims_reconsturction(0,feats_epshn,computer_similarity_method,maps_epshn,idx,sortedInds_epshn_furthest,labels,0,0)
    img0 = combine_vert([np.array(epshn_epshn_furthest_imgs),np.array(epshn_furthest_map)])
    
    epshn_epshn_nearest_imgs = combine_horz([np.zeros((256,256,3)),np.array(combine_horz([np.array(img_with_txt('%0.5f'%(d))) for d in dists_epshn[sortedInds_epshn_nearest]]))])
    epshn_nearest_map, s1_arcface, s2_arcface = make_sim_ims_reconsturction(0,feats_epshn,computer_similarity_method,maps_epshn,idx,sortedInds_epshn_nearest,labels,0,0)
    img1 = combine_vert([np.array(epshn_epshn_nearest_imgs),np.array(epshn_nearest_map)])
    
    epshn_epshn_samefurthest_imgs = combine_horz([np.zeros((256,256,3)),np.array(combine_horz([np.array(img_with_txt('%0.5f'%(d))) for d in dists_epshn[sortedInds_epshn_same_furthest]]))])
    epshn_samefurthest_map, s1_arcface, s2_arcface = make_sim_ims_reconsturction(0,feats_epshn,computer_similarity_method,maps_epshn,idx,sortedInds_epshn_same_furthest,labels,0,0)
    img2 = combine_vert([np.array(epshn_epshn_samefurthest_imgs),np.array(epshn_samefurthest_map)])
    
    outImg = combine_vert([np.array(img0),np.zeros((10,epshn_epshn_furthest_imgs.width)),np.array(img1),np.zeros((10,epshn_epshn_furthest_imgs.width)),np.array(img2)])
    outImg.save(outFilename)
    print(outFilename)

In [ ]:
import importlib
import sys
importlib.reload(sys.modules['similarity_ops'])
from similarity_ops import *


numResults = 20
computer_similarity_method = SVD_reconstruction_thres

output_folder = os.path.join(Data,'test_pair_consturction_14*14',tra_or_val)
#output_folder = os.path.join(Data,tra_or_val,'top_results')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

epshn_maps = {}
#arcface_maps = {}
for idx in range(0,5000,10):
#for idx in range(0,5000,10):
    epshn_maps[idx] = []
    #arcface_maps[idx] = []
    
    epshn_feat = feats_epshn[idx]
    #arcface_feat = feats_arcface[idx]
    queryCls = labels[idx]
    sameClassInds = np.where(labels==queryCls)[0]
    outFilename = os.path.join(output_folder,str(idx)+'.jpg')

#     # arcface dists
#     dists_arcface = [np.dot(arcface_feat/np.linalg.norm(arcface_feat),f/np.linalg.norm(f)) for f in feats_arcface]
#     dists_arcface = np.array(dists_arcface)
#     sortedInds_arcface = np.argsort(-dists_arcface)[:numResults]

    # epshn dists
    dists_epshn = [np.dot(epshn_feat/np.linalg.norm(epshn_feat),f/np.linalg.norm(f)) for f in feats_epshn]
    dists_epshn = np.array(dists_epshn)
    sortedInds_epshn= np.argsort(-dists_epshn)
    

    sortedInds_epshn_furthest = sortedInds_epshn[-numResults:]
    sortedInds_epshn_nearest = sortedInds_epshn[:numResults]
    sortedInds_epshn_same_furthest = sortedInds_epshn[labels[sortedInds_epshn] == labels[idx]][-10:]
    
    epshn_epshn_furthest_imgs = combine_horz([np.zeros((256,256,3)),np.array(combine_horz([np.array(img_with_txt('%0.5f'%(d))) for d in dists_epshn[sortedInds_epshn_furthest]]))])
    epshn_furthest_map, s1_arcface, s2_arcface = make_sim_ims_reconsturction(0,feats_epshn,computer_similarity_method,maps_epshn,idx,sortedInds_epshn_furthest,labels,0,0)
    img0 = combine_vert([np.array(epshn_epshn_furthest_imgs),np.array(epshn_furthest_map)])
    
    epshn_epshn_nearest_imgs = combine_horz([np.zeros((256,256,3)),np.array(combine_horz([np.array(img_with_txt('%0.5f'%(d))) for d in dists_epshn[sortedInds_epshn_nearest]]))])
    epshn_nearest_map, s1_arcface, s2_arcface = make_sim_ims_reconsturction(0,feats_epshn,computer_similarity_method,maps_epshn,idx,sortedInds_epshn_nearest,labels,0,0)
    img1 = combine_vert([np.array(epshn_epshn_nearest_imgs),np.array(epshn_nearest_map)])
    
    epshn_epshn_samefurthest_imgs = combine_horz([np.zeros((256,256,3)),np.array(combine_horz([np.array(img_with_txt('%0.5f'%(d))) for d in dists_epshn[sortedInds_epshn_same_furthest]]))])
    epshn_samefurthest_map, s1_arcface, s2_arcface = make_sim_ims_reconsturction(0,feats_epshn,computer_similarity_method,maps_epshn,idx,sortedInds_epshn_same_furthest,labels,0,0)
    img2 = combine_vert([np.array(epshn_epshn_samefurthest_imgs),np.array(epshn_samefurthest_map)])
    
    outImg = combine_vert([np.array(img0),np.zeros((10,epshn_epshn_furthest_imgs.width)),np.array(img1),np.zeros((10,epshn_epshn_furthest_imgs.width)),np.array(img2)])
    outImg.save(outFilename)
    print(outFilename)

In [16]:
def embed_resnet(dsets,whichModel,whichConv):
    whichModel.train(False)
    dataLoader = torch.utils.data.DataLoader(dsets, batch_size=10, sampler=SequentialSampler(dsets), num_workers=12)
    # iterate batch
    V,M,L = [],[],[]
    modules=list(whichModel.children())[:-2]
    whichModel_pool = nn.Sequential(*modules)
    whichModel_pool = whichModel_pool.cuda()
    for data in dataLoader:
        with torch.set_grad_enabled(False):
            inputs_bt, labels_bt = data['data'], data['label']  # <FloatTensor> <LongTensor>
            fmap_tmp = whichModel_pool(inputs_bt.cuda())
            #fvec = whichModel_fc(fmap_tmp)
            #fvec = whichModel(inputs_bt.cuda())
            # i need a tensor norml2 function here
#             fmap = norml2(fc_Conv(fmap_tmp))
#             fmap = whichConv(fmap_tmp)
            fmap = fmap_tmp
            fvec = whichModel(inputs_bt.cuda())
            
        V.extend(fvec.cpu().numpy())
        M.extend(fmap.cpu().numpy())
        L.extend(labels_bt.numpy())
        
    maps_epshn = np.moveaxis(np.array(M),1,3)
    feats_epshn = np.array(V)
    labels = np.array(L)
    for parameter in whichModel.fc.parameters():
        epshn_weighs = parameter
        break
    epshn_weighs=epshn_weighs.cpu().detach().numpy()
    return feats_epshn, maps_epshn, labels, epshn_weighs

In [20]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import PIL
data_dict = torch.load('../googlelandmark_epshn/data_dict_landmark.pth')
model = torch.load('../googlelandmark_epshn/_result/EPSHN/GoogleLandmark_R101/512/G16/0/model.pth')
#model = torch.load('../googlelandmark_epshn/_result/EPSHN/Landmark_R101/512/G16/0/')
data_dict_val = data_dict['val']
new_data_dict_val = {}
for idx,key in enumerate(list(data_dict_val.keys())):
    if idx < 300:
        pass
    if idx > 400:
        break
    new_data_dict_val[key] = data_dict_val[key]
data_dict['val'] = new_data_dict_val

data_transforms = transforms.Compose([transforms.Resize(512),
                                      transforms.CenterCrop(512),
                                      transforms.ToTensor()
                                      ])

dsets = ImageReader(data_dict['val'], data_transforms)

data_transforms2 = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(256)])
dsets2 = ImageReader(data_dict['val'], data_transforms2)

In [21]:
model = model.cuda()
feats_epshn, maps_epshn, labels, epshn_weighs = embed_resnet(dsets,model,0)

In [22]:
feats_epshn.shape

(37274, 512)

In [25]:
Data = 'GoogleLandmark'
tra_or_val = 'val'

In [ ]:
import importlib
import sys
importlib.reload(sys.modules['similarity_ops'])
from similarity_ops import *

numResults = 20
computer_similarity_method = SVD_whole

output_folder = os.path.join(Data,'SVD_wholeclass_16*16',tra_or_val)
#output_folder = os.path.join(Data,tra_or_val,'top_results')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

epshn_maps = {}
#arcface_maps = {}
for idx in set(labels):
    sameClassInds = np.where(labels==idx)[0]
    outFilename = os.path.join(output_folder,str(idx)+'.jpg')
    
    res_map= make_sim_ims_svdwhole(feats_epshn,computer_similarity_method,maps_epshn,sameClassInds,labels,0,0)
    res_map.save(outFilename)
    print(outFilename)

In [31]:
import importlib
import sys
importlib.reload(sys.modules['similarity_ops'])
from similarity_ops import *


numResults = 20
computer_similarity_method = SVD_reconstruction_thres

output_folder = os.path.join(Data,'pair_consturction_16*16','val')
#output_folder = os.path.join(Data,tra_or_val,'top_results')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

epshn_maps = {}
#arcface_maps = {}
for idx in range(0,500,10):
#for idx in range(0,5000,10):
    epshn_maps[idx] = []
    #arcface_maps[idx] = []
    
    epshn_feat = feats_epshn[idx]
    #arcface_feat = feats_arcface[idx]
    queryCls = labels[idx]
    sameClassInds = np.where(labels==queryCls)[0]
    outFilename = os.path.join(output_folder,str(idx)+'.jpg')

#     # arcface dists
#     dists_arcface = [np.dot(arcface_feat/np.linalg.norm(arcface_feat),f/np.linalg.norm(f)) for f in feats_arcface]
#     dists_arcface = np.array(dists_arcface)
#     sortedInds_arcface = np.argsort(-dists_arcface)[:numResults]

    # epshn dists
    dists_epshn = [np.dot(epshn_feat/np.linalg.norm(epshn_feat),f/np.linalg.norm(f)) for f in feats_epshn]
    dists_epshn = np.array(dists_epshn)
    sortedInds_epshn= np.argsort(-dists_epshn)
    

    sortedInds_epshn_furthest = sortedInds_epshn[-numResults:]
    sortedInds_epshn_nearest = sortedInds_epshn[:numResults]
    sortedInds_epshn_same_furthest = sortedInds_epshn[labels[sortedInds_epshn] == labels[idx]][-10:]
    
    epshn_epshn_furthest_imgs = combine_horz([np.zeros((256,256,3)),np.array(combine_horz([np.array(img_with_txt('%0.5f'%(d))) for d in dists_epshn[sortedInds_epshn_furthest]]))])
    epshn_furthest_map, s1_arcface, s2_arcface = make_sim_ims_reconsturction(epshn_weighs,feats_epshn,computer_similarity_method,maps_epshn,idx,sortedInds_epshn_furthest,labels,0,0)
    img0 = combine_vert([np.array(epshn_epshn_furthest_imgs),np.array(epshn_furthest_map)])
    
    epshn_epshn_nearest_imgs = combine_horz([np.zeros((256,256,3)),np.array(combine_horz([np.array(img_with_txt('%0.5f'%(d))) for d in dists_epshn[sortedInds_epshn_nearest]]))])
    epshn_nearest_map, s1_arcface, s2_arcface = make_sim_ims_reconsturction(epshn_weighs,feats_epshn,computer_similarity_method,maps_epshn,idx,sortedInds_epshn_nearest,labels,0,0)
    img1 = combine_vert([np.array(epshn_epshn_nearest_imgs),np.array(epshn_nearest_map)])
    
    epshn_epshn_samefurthest_imgs = combine_horz([np.zeros((256,256,3)),np.array(combine_horz([np.array(img_with_txt('%0.5f'%(d))) for d in dists_epshn[sortedInds_epshn_same_furthest]]))])
    epshn_samefurthest_map, s1_arcface, s2_arcface = make_sim_ims_reconsturction(epshn_weighs,feats_epshn,computer_similarity_method,maps_epshn,idx,sortedInds_epshn_same_furthest,labels,0,0)
    img2 = combine_vert([np.array(epshn_epshn_samefurthest_imgs),np.array(epshn_samefurthest_map)])
    
    outImg = combine_vert([np.array(img0),np.zeros((10,epshn_epshn_furthest_imgs.width)),np.array(img1),np.zeros((10,epshn_epshn_furthest_imgs.width)),np.array(img2)])
    outImg.save(outFilename)
    print(outFilename)

/pless_nfs/home/liuxiaotong2017/Comparison_vis/similarity_ops.py:1086: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  temp_1,resid,rank,s = np.linalg.lstsq(v2[0:n_dim].T,conv1.T)
/pless_nfs/home/liuxiaotong2017/Comparison_vis/similarity_ops.py:1110: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  temp_2,resid,rank,s = np.linalg.lstsq(v1[0:n_dim].T,conv2.T)


GoogleLandmark/pair_consturction_16*16/val/0.jpg
GoogleLandmark/pair_consturction_16*16/val/10.jpg
GoogleLandmark/pair_consturction_16*16/val/20.jpg
GoogleLandmark/pair_consturction_16*16/val/30.jpg
GoogleLandmark/pair_consturction_16*16/val/40.jpg
GoogleLandmark/pair_consturction_16*16/val/50.jpg
GoogleLandmark/pair_consturction_16*16/val/60.jpg
GoogleLandmark/pair_consturction_16*16/val/70.jpg
GoogleLandmark/pair_consturction_16*16/val/80.jpg
GoogleLandmark/pair_consturction_16*16/val/90.jpg
GoogleLandmark/pair_consturction_16*16/val/100.jpg
GoogleLandmark/pair_consturction_16*16/val/110.jpg
GoogleLandmark/pair_consturction_16*16/val/120.jpg
GoogleLandmark/pair_consturction_16*16/val/130.jpg
GoogleLandmark/pair_consturction_16*16/val/140.jpg
GoogleLandmark/pair_consturction_16*16/val/150.jpg
GoogleLandmark/pair_consturction_16*16/val/160.jpg
GoogleLandmark/pair_consturction_16*16/val/170.jpg
GoogleLandmark/pair_consturction_16*16/val/180.jpg
GoogleLandmark/pair_consturction_16*16/val

In [32]:
numResults = 20
computer_similarity_method = combine_SVD

output_folder = os.path.join(Data,'combine_SVD_16*16',tra_or_val)
#output_folder = os.path.join(Data,tra_or_val,'top_results')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

epshn_maps = {}
#arcface_maps = {}
for idx in range(0,500,10):
#for idx in range(0,5000,10):
    epshn_maps[idx] = []
    #arcface_maps[idx] = []
    
    epshn_feat = feats_epshn[idx]
    #arcface_feat = feats_arcface[idx]
    queryCls = labels[idx]
    sameClassInds = np.where(labels==queryCls)[0]
    outFilename = os.path.join(output_folder,str(idx)+'.jpg')

#     # arcface dists
#     dists_arcface = [np.dot(arcface_feat/np.linalg.norm(arcface_feat),f/np.linalg.norm(f)) for f in feats_arcface]
#     dists_arcface = np.array(dists_arcface)
#     sortedInds_arcface = np.argsort(-dists_arcface)[:numResults]

    # epshn dists
    dists_epshn = [np.dot(epshn_feat/np.linalg.norm(epshn_feat),f/np.linalg.norm(f)) for f in feats_epshn]
    dists_epshn = np.array(dists_epshn)
    sortedInds_epshn= np.argsort(-dists_epshn)
    

    sortedInds_epshn_furthest = sortedInds_epshn[-numResults:]
    sortedInds_epshn_nearest = sortedInds_epshn[:numResults]
    sortedInds_epshn_same_furthest = sortedInds_epshn[labels[sortedInds_epshn] == labels[idx]][-10:]
    
    epshn_epshn_furthest_imgs = combine_horz([np.zeros((256,256,3)),np.array(combine_horz([np.array(img_with_txt('%0.5f'%(d))) for d in dists_epshn[sortedInds_epshn_furthest]]))])
    epshn_furthest_map, s1_arcface, s2_arcface = make_sim_ims_rgb(0,feats_epshn,computer_similarity_method,maps_epshn,idx,sortedInds_epshn_furthest,labels,0,0)
    img0 = combine_vert([np.array(epshn_epshn_furthest_imgs),np.array(epshn_furthest_map)])
    
    epshn_epshn_nearest_imgs = combine_horz([np.zeros((256,256,3)),np.array(combine_horz([np.array(img_with_txt('%0.5f'%(d))) for d in dists_epshn[sortedInds_epshn_nearest]]))])
    epshn_nearest_map, s1_arcface, s2_arcface = make_sim_ims_rgb(0,feats_epshn,computer_similarity_method,maps_epshn,idx,sortedInds_epshn_nearest,labels,0,0)
    img1 = combine_vert([np.array(epshn_epshn_nearest_imgs),np.array(epshn_nearest_map)])
    
    epshn_epshn_samefurthest_imgs = combine_horz([np.zeros((256,256,3)),np.array(combine_horz([np.array(img_with_txt('%0.5f'%(d))) for d in dists_epshn[sortedInds_epshn_same_furthest]]))])
    epshn_samefurthest_map, s1_arcface, s2_arcface = make_sim_ims_rgb(0,feats_epshn,computer_similarity_method,maps_epshn,idx,sortedInds_epshn_same_furthest,labels,0,0)
    img2 = combine_vert([np.array(epshn_epshn_samefurthest_imgs),np.array(epshn_samefurthest_map)])
    
    outImg = combine_vert([np.array(img0),np.zeros((10,epshn_epshn_furthest_imgs.width)),np.array(img1),np.zeros((10,epshn_epshn_furthest_imgs.width)),np.array(img2)])
    outImg.save(outFilename)
    print(outFilename)

GoogleLandmark/combine_SVD_16*16/val/0.jpg
GoogleLandmark/combine_SVD_16*16/val/10.jpg
GoogleLandmark/combine_SVD_16*16/val/20.jpg
GoogleLandmark/combine_SVD_16*16/val/30.jpg
GoogleLandmark/combine_SVD_16*16/val/40.jpg
GoogleLandmark/combine_SVD_16*16/val/50.jpg
GoogleLandmark/combine_SVD_16*16/val/60.jpg
GoogleLandmark/combine_SVD_16*16/val/70.jpg
GoogleLandmark/combine_SVD_16*16/val/80.jpg
GoogleLandmark/combine_SVD_16*16/val/90.jpg
GoogleLandmark/combine_SVD_16*16/val/100.jpg
GoogleLandmark/combine_SVD_16*16/val/110.jpg
GoogleLandmark/combine_SVD_16*16/val/120.jpg
GoogleLandmark/combine_SVD_16*16/val/130.jpg
GoogleLandmark/combine_SVD_16*16/val/140.jpg
GoogleLandmark/combine_SVD_16*16/val/150.jpg
GoogleLandmark/combine_SVD_16*16/val/160.jpg
GoogleLandmark/combine_SVD_16*16/val/170.jpg
GoogleLandmark/combine_SVD_16*16/val/180.jpg
GoogleLandmark/combine_SVD_16*16/val/190.jpg
GoogleLandmark/combine_SVD_16*16/val/200.jpg
GoogleLandmark/combine_SVD_16*16/val/210.jpg
GoogleLandmark/combin